In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

import time
from datetime import datetime
import pandas as pd
import numpy as np
import json

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [ ]:
import re
from pyspark.sql.types import StringType, TimestampType, FloatType

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Greta") \
    .config("spark.sql.debug.maxToStringFields", 500) \
    .getOrCreate()
spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
spark.conf.set("spark.executor.memory","4g")
spark.conf.set("spark.driver.memory","4g")
spark.conf.set("spark.driver.maxResultSize","4g")

In [ ]:
# Read JSON file into dataframe
df = spark.read.json("hdfs://localhost:9000/ca2/Greta/greta.ndjson")

In [ ]:
df.printSchema()
df.show()

In [ ]:
df.count()

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df.select("created_at","full_text").show(10, truncate=True)

In [ ]:
df.select("created_at","full_text").show(1, truncate=False)

# START FROM BEGINNING

In [ ]:
df_work = df.select("created_at","full_text")

In [ ]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'greta', ' ', text)
    text = re.sub(r'thunberg', ' ', text)
    text = re.sub(r'@[a-zA-Z0-9_]+', ' ', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', ' ', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', ' ', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub(' +', ' ',text)
    text = [token for token in text.split() if len(token) > 2]
    text = [token for token in text if token]
    text = ' '.join(text)
    return text

In [ ]:
cleanUDF = udf(lambda x:clean_text(x),StringType())

In [ ]:
df_work = df_work.withColumn('Text', cleanUDF(F.col('full_text')))

In [ ]:
def createTimestamp(created_at):
    newTimestamp = datetime.strftime(datetime.strptime
                                     (created_at,
                                      '%a %b %d %H:%M:%S +0000 %Y'),
                                    '%Y-%m-%d %H:%M:%S')
    return newTimestamp

In [ ]:

timestampUDF = udf(lambda x:createTimestamp(x),StringType())

In [ ]:
df_work = df_work.withColumn('Timestamp', timestampUDF(F.col('created_at')))

In [ ]:
df_work = df_work.drop('created_at')

In [ ]:
df_work = df_work.drop('full_text')

In [ ]:
df_work = df_work.filter(F.col('Text').rlike('[a-zA-Z]'))

In [ ]:
df_work.show()

In [ ]:
from pyspark.sql.types import LongType, StructField, StructType

new_schema = StructType([StructField('rowId',LongType(),True)]
                        + df_work.schema.fields)
zip_rdd = df_work.rdd.zipWithIndex()

In [ ]:
new_rdd = zip_rdd.map(lambda args: ([args[1]+1] + list(args[0])))

In [ ]:
df_work = spark.createDataFrame(new_rdd,new_schema)

In [ ]:
df_work.show(5)

In [ ]:
df_work = df_work.where(df_work.rowId%10==0)

In [ ]:
df_work.count()

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
blob = df_work.select('Text').toPandas()

In [ ]:
wc = WordCloud(background_color='white',
                    stopwords =  set(STOPWORDS),
                    max_words = 50, 
                    random_state = 42,)
wc.generate(' '.join(blob['Text']))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wc)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
def polarity(text):
    text = analyser.polarity_scores(text)['compound']
    return text

In [ ]:

polarityUDF = udf(lambda x:polarity(x),FloatType())

In [ ]:
df_work = df_work.withColumn('polarity', polarityUDF(F.col('Text')))

In [ ]:
df_work.show(5)